# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [2]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [3]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

In [25]:
def split_data(chars, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    split_frac: Fraction of batches to keep in the training set 
    
    """
    
    # Split into training and validation sets, keep the first split_frac batches for training
    split_idx = int(len(chars)*split_frac)
    print(split_idx)
    X_train= chars[:split_idx]
    X_val  = chars[split_idx:]
    
    return np.array(X_train), np.array(X_val)

def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * characters_per_batch]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [48]:
X_train, X_val = split_data(encoded, 0.9)

batches = get_batches(X_train, 100, 100)
x, y = next(batches)
print('encoded:', len(encoded), 'X_train:', len(X_train),
      'X_val', len(X_val), 'batches:', len(list(batches)), 'x shape:', x.shape)

1822937
('encoded:', 2025486, 'X_train:', 1822937, 'X_val', 202549, 'batches:', 181, 'x shape:', (100, 100))


### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

In [5]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = self.build_inputs(batch_size, num_steps)
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes, name='x_one_hot')
        
        # Build the LSTM cell
        cell, self.initial_state = self.build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)
        
        # Run each sequence step through the RNN and collect the outputs
        # dynamic_rnn receive Tensor (batch, steps, inputs) or (steps, batch, inputs) as input.
        # output is a tensor of shape [batch_size, max_time, lstm_size]
        # Make sure the time_major is changed accordingly.
        with tf.name_scope("RNN_forward"):
            outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = self.build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = self.build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = self.build_optimizer(self.loss, learning_rate, grad_clip)
        self.merged = tf.summary.merge_all()
       
    def build_inputs(self, batch_size, num_steps):
        ''' Define placeholders for inputs, targets, and dropout 
        
            Arguments
            ---------
            batch_size: Batch size, number of sequences per batch
            num_steps: Number of sequence steps in a batch
            
        '''
        # Declare placeholders we'll feed into the graph
        with tf.name_scope('input'):
            inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
            
        with tf.name_scope('target'):
            targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')                    
        
        # Keep probability placeholder for drop out layers
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
        return inputs, targets, keep_prob
        
    def build_lstm(self, lstm_size, num_layers, batch_size, keep_prob):
        ''' Build LSTM cell.
        
            Arguments
            ---------
            keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
            lstm_size: Size of the hidden layers in the LSTM cells
            num_layers: Number of LSTM layers
            batch_size: Batch size
    
        '''
        ### Build the LSTM Cell
        with tf.name_scope("RNN_cells"):
            stacked_rnn = []
            for _ in range(3):
                # Use a basic LSTM cell
                lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)    
                # Add dropout to the cell
                drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
                stacked_rnn.append(drop)
            # Stack up multiple LSTM cells into layers, for deep learning
            cell = tf.contrib.rnn.MultiRNNCell(stacked_rnn)
            
        with tf.name_scope('RNN_init_state'):
            initial_state = cell.zero_state(batch_size, tf.float32)
        
        return cell, initial_state
    
    # connect the output of the RNN cells to a full connected layer with a softmax output. 
    # The softmax output gives us a probability distribution we can use to predict the next character
    # If our input has batch size $N$, number of steps $M$, and the hidden layer has L hidden units,
    # then the output is a 3D tensor with size N x M x L. The output of each LSTM cell has
    # size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total
    # size is N x M x L.
    # We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make
    # things easier, we should reshape the outputs into a 2D tensor with shape (M * N) x L. That is, 
    # one row for each sequence and step, where the values of each row are the output from the LSTM cells
    
    def build_output(self, lstm_output, in_size, out_size):
        ''' Build a softmax layer, return the softmax output and logits.
        
            Arguments
            ---------
            
            x: Input tensor
            in_size: Size of the input tensor, for example, size of the LSTM cells (lstm_size)
            out_size: Size of this softmax layer (number_classes)
        
        '''
    
        # Reshape output so it's a bunch of rows, one row for each step for each sequence.
        # That is, the shape should be batch_size*num_steps rows by lstm_size columns
        with tf.name_scope('sequence_reshape'):
            seq_output = tf.concat(lstm_output, axis=1)
            x = tf.reshape(seq_output, [-1, in_size])
        
        # Connect the RNN outputs to a softmax layer
        with tf.name_scope('logits'):
            softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1), name='softmax_w')
            softmax_b = tf.Variable(tf.zeros(out_size), name='softmax_b')        
            # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
            # of rows of logit outputs, one for each step and sequence
            logits = tf.matmul(x, softmax_w) + softmax_b
            tf.summary.histogram('softmax_w', softmax_w)
            tf.summary.histogram('softmax_b', softmax_b)
            
        # Use softmax to get the probabilities for predicted characters
        with tf.name_scope('predictions'):
            preds = tf.nn.softmax(logits, name='predictions')
            tf.summary.histogram('predictions', preds)
        
        return preds, logits

    # Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss.
    # First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the
    # one-hot targets so it's a 2D tensor with size(M∗N)×C where C is the number of classes/characters
    # we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with C units.
    # So our logits will also have size (M∗N)×C .
    
    def build_loss(self, logits, targets, lstm_size, num_classes):
        ''' Calculate the loss from the logits and the targets.
        
            Arguments
            ---------
            logits: Logits from final fully connected layer
            targets: Targets for supervised learning
            lstm_size: Number of LSTM hidden units
            num_classes: Number of classes in targets
            
        '''
        # One-hot encode targets and reshape to match logits, one row per batch_size per step
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
            
        # Softmax cross entropy loss
        with tf.name_scope('loss'):
            cost = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='cost')
            loss = tf.reduce_mean(cost, name='loss')
            tf.summary.scalar('loss', loss)
        return loss

    # Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing.
    # LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this,
    # we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold,
    # we set it to the threshold. This will ensure the gradients never grow overly large.
    
    def build_optimizer(self, loss, learning_rate, grad_clip):
        ''' Build optmizer for training, using gradient clipping.
        
            Arguments:
            loss: Network loss
            learning_rate: Learning rate for optimizer
        
        '''
        
        # Optimizer for training, using gradient clipping to control exploding gradients
        with tf.name_scope('train'):
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
            train_op = tf.train.AdamOptimizer(learning_rate)
            optimizer = train_op.apply_gradients(zip(grads, tvars))
        
        return optimizer

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

In [49]:
def train(model, epochs, train_filewriter, test_filewriter):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Use the line below to load a checkpoint and resume training
        #saver.restore(sess, 'checkpoints/______.ckpt')

        characters_per_batch = batch_size * num_steps
        n_batches = int(len(encoded)/characters_per_batch)
        print(n_batches)
        # Start training iterations
        for e in range(epochs):
            # Train network
            new_state = sess.run(model.initial_state)
            loss = 0
            cnt = 0
            for x, y in get_batches(X_train, batch_size, num_steps): 
                cnt += 1
                iteration = e*n_batches + cnt
                #start = time.time()
                feed = {model.inputs: x,
                        model.targets: y,
                        model.keep_prob: keep_prob,
                        model.initial_state: new_state}
                summary, batch_loss, new_state, _ = sess.run([model.merged,
                                                              model.loss, 
                                                              model.final_state, 
                                                              model.optimizer], 
                                                              feed_dict=feed) 
                loss += batch_loss
                
                # write training logs
                train_writer.add_summary(summary, iteration)
                
                # Validation:
                if (iteration%save_every_n == 0):
                    # Check performance, notice dropout has been set to 1
                    val_loss = []
                    new_state = sess.run(model.initial_state)
                    for x, y in get_batches(X_val, batch_size, num_steps):
                        feed = {model.inputs: x,
                                model.targets: y,
                                model.keep_prob: 1.,
                                model.initial_state: new_state}
                        summary, batch_loss, new_state = sess.run([model.merged, 
                                                                   model.loss, 
                                                                   model.final_state],
                                                                   feed_dict=feed)
                        val_loss.append(batch_loss)
                    # write val logs   
                    test_writer.add_summary(summary, iteration)
                    #end = time.time()
                    print('Epoch: {}/{}... '.format(e+1, epochs),
                          'Training Step: {}... '.format(iteration),
                          'Training/Val loss: {:.4f}/{:.4f}... '.format(loss/cnt, np.mean(val_loss)))
                          #'{:.4f} sec/200 batches... '.format((end-start)))
                    # save checkpoint
                    #saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(iteration, lstm_size))
                
            
    #saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

In [50]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

epochs = 10
save_every_n = 50      #

for lstm_size in [128, 256]:
    for num_layers in [2]:
        for learning_rate in [0.001]:
            log_path = './logs/rnn/lr={},rl={},ru={}'.format(learning_rate, num_layers, lstm_size)
            
            model = CharRNN(len(vocab), 
                            batch_size=batch_size,
                            num_steps=num_steps,                            
                            lstm_size=lstm_size,
                            num_layers=num_layers, 
                            learning_rate=learning_rate)
            
            train_writer = tf.summary.FileWriter(log_path + '/train', tf.get_default_graph())
            test_writer = tf.summary.FileWriter(log_path + '/test')
            
            train(model, epochs, train_writer, test_writer)

202
('Epoch: 1/10... ', 'Training Step: 50... ', 'Training/Val loss: 3.4890/3.1773... ')
('Epoch: 1/10... ', 'Training Step: 100... ', 'Training/Val loss: 3.3520/3.1652... ')
('Epoch: 1/10... ', 'Training Step: 150... ', 'Training/Val loss: 3.2969/3.1621... ')
('Epoch: 2/10... ', 'Training Step: 250... ', 'Training/Val loss: 3.1638/3.1475... ')
('Epoch: 2/10... ', 'Training Step: 300... ', 'Training/Val loss: 3.1308/2.9646... ')
('Epoch: 2/10... ', 'Training Step: 350... ', 'Training/Val loss: 3.0471/2.6933... ')
('Epoch: 3/10... ', 'Training Step: 450... ', 'Training/Val loss: 2.6102/2.4898... ')
('Epoch: 3/10... ', 'Training Step: 500... ', 'Training/Val loss: 2.5703/2.4345... ')
('Epoch: 3/10... ', 'Training Step: 550... ', 'Training/Val loss: 2.5408/2.3953... ')
('Epoch: 4/10... ', 'Training Step: 650... ', 'Training/Val loss: 2.4219/2.3379... ')
('Epoch: 4/10... ', 'Training Step: 700... ', 'Training/Val loss: 2.4043/2.3136... ')
('Epoch: 4/10... ', 'Training Step: 750... ', 'Trai

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [9]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [10]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [11]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Love")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i4040_l512.ckpt
Loved and
shought out of the secule side of the distance in the places and
came and started through his below.

"I don't know that," said Vronsky, when he was a post of that's something,
when she had said how we who had been a sense of children.

"Yes, you'd not believe it."

Then he saw all of the sick man, took to her happiness.

"I'm now, but I cannot centry at the cros."

She seemed a good hearted little same and continually done where they
set to speak about himself. And he could not live had an instant, and was
not strange in the peasant whom he had not said:

"There's sumble anything."

"I would be something, though I couldn't be disturbed to see her in
the peasants about him to them. And how have you to stay to her than you there?
There is the preterved and decision in the day only it to be taken up, and
I won't care for him."

"Yes, I so a less of service."

"I shall be done, and there's no one in the doctor